In [10]:
from itertools import product, combinations
import math
from pathlib import Path
import pandas as pd
import torch
from plotnine import *
import magicsoup as ms
import magicsoup.examples.wood_ljungdahl
import magicsoup.examples.reverse_krebs

theme_set(theme_minimal())
this_dir = Path(".") / "experiments" / "e1_co2_fixing"
world = ms.World.from_file(filepath=this_dir / "runs" / "2023-01-09_19-39" / "world.pkl")

TypeError: __new__() missing 2 required positional arguments: 'name' and 'energy'